# First order differential equations

In the following, we solve the following 1st order inhomogeneous differential equation using Python's scipy Library. Let $A\in \mathbb{C}^{n \times n}$ with $n\in \mathbb{N}$

$\begin{align}
\frac{\partial^{n} }{\partial t^{n} }x(t) = A \, x(t) + u(t)\end{align}$

## Homogeneous part

The following equation denotes the homogeneous part of the equation:

$\begin{align}
\frac{\partial^{n} }{\partial t^{n} }x(t) = A \, x(t) \\

\end{align}$ 

And the following code solves it:

### Loading Necessary Modules
This code snippet loads essential Python libraries needed for our calculations.

In [1]:
# Load the modules
import numpy as np
from scipy.linalg import eig
import sympy as sp

### Defining and Analyzing the System
Define the system matrix and analyze it by calculating eigenvalues and eigenvectors.

In [2]:
# Homogeneous Solution
def analyse_system(A):
    t = sp.symbols('t', real=True)                  # Define symbolic time
    eigenvalues, eigenvectors = eig(A, right=True)  # Calculate Eigenvalues and -vectors 
    
    # Adjust eigenvectors to scale the first non-zero element to 1
    for i in range(eigenvectors.shape[1]):
        vec = eigenvectors[:, i]
        non_zero_index = np.nonzero(vec)[0][0]      # Find the first non-zero index
        eigenvectors[:, i] /= vec[non_zero_index]   # Scale by the first non-zero element

    # Handle multiplicity
    unique_eigenvalues, unique_indices, counts = np.unique(eigenvalues, return_index=True, return_counts=True)
    
    print('Eigenvalues and their eigenvectors, respectively:')
    ci_count = 1  # Counter
    solution_terms = []
    stability = 'stable'
    constant_terms = []  # To store constant descriptions
    for i, eigenvalue in enumerate(unique_eigenvalues):
        multiplicity = counts[i]
        lambda_real = round(np.real(eigenvalue), 3)  # Use numpy.round for the real part of eigenvalue
        eigenvector = eigenvectors[:, unique_indices[i]]
        rounded_eigenvector = tuple(np.round(np.real(v), 3) for v in eigenvector)  # Round every component in the eigenvector

        print(f'Egenvalue {i+1} ({np.round(eigenvalue, 3)}): Multiplicity {multiplicity}, Eigenvektor {i+1}: Vector{rounded_eigenvector}')

        if lambda_real > 0:
            stability = 'unstable'
        elif lambda_real == 0:
            if stability != 'unstable':
                stability = 'stable, but not asymptotically stable'
        else:
            if stability != 'unstable' and stability != 'stable, but not asymptotically stable':
                stability = 'asymptotically stable'

        for j in range(multiplicity):
            ci = sp.symbols(f'c{ci_count}', real=True)
            t_power = f't**{j}' if j > 1 else ('t' if j == 1 else '')
            term = f'{ci} * Vector{rounded_eigenvector} * {t_power} * exp({lambda_real} * t)'
            term = term.replace(' * ', ' ').strip()
            solution_terms.append(term)
            constant_terms.append(f'c{ci_count} (real number)')  # Add constant description
            ci_count += 1  # Increment counter for each constant

    # Output the homogeneous solution
    solution_string = ' + '.join(solution_terms).replace('  ', ' ')
    constants_description = ', '.join(constant_terms)  # Describe the constants
    print(f'Homogeneous solution: y_h(t) = {solution_string}')
    print(f'Constants: {constants_description}')  # Print constants as real numbers
    print()  # Newline
    print(f'Stability: The system is {stability}.')

# Analyse the system
analyse_system(A)

Eigenvalues and their eigenvectors, respectively: 
Eigenvalue 1 ((-1+0j)): Multiplicity 3, Eigenvektor 1: Vector(1.0, 0.0, 0.0)

Homogeneous solution: y_h(t) = c1 Vector(1.0, 0.0, 0.0) exp(-1.0 t) + c2 Vector(1.0, 0.0, 0.0) t exp(-1.0 t) + c3 Vector(1.0, 0.0, 0.0) t**2 exp(-1.0 t) 
Constants: c1 (real number), c2 (real number), c3 (real number) 

Stability: The system is asymptotically stable.


Hence the homogenous solution is

$\begin{align}
y_{h} = c_{1} \, \begin{pmatrix} 1 \\ 2 \\ 3 end{pmatrix} \exp\left-t\right) + c_{2} \, \exp\left-t\right) + c_{3} \, \exp\left-t\right), \quad c_{1}, \, c_{2}, \, \in \mathbb{R} 

\end{align}$ 

Which was what we want..

## Inhomogeneous part
This part solves the inhomogeneous part of the equation.

which was to be shown.